# Deep Q-Network (DQN)
---
In this notebook, you will implement a DQN agent with OpenAI Gym's LunarLander-v2 environment.

### 1. Import the Necessary Packages

In [ ]:
import gymnasium as gym
import random
import torch
import numpy as np
import time
from collections import deque
import matplotlib.pyplot as plt
%matplotlib inline

is_ipython = 'inline' in plt.get_backend()
if is_ipython:
    print("Ipython")
    from IPython import display
else:
    print("pyvirtualdisplay")
    #!python -m pip install pyvirtualdisplay
    from pyvirtualdisplay import Display
    display = Display(visible=0, size=(1400, 900))
    display.start()
ion_context = plt.ion()

from DQN_agent import *
from DQN_utils import *
print("Using device", device, "imported from DQN_utils.")

### 2. Instantiate the Environment and Agent

Initialize the environment in the code cell below.

In [ ]:
##### NEW GYM = GYMNASIUM
#!pip install gymnasium[box2d]
#import gymnasium as gym
env = gym.make("LunarLander-v2", render_mode="rgb_array",    #"human",       #
                                 continuous= False,
                                 gravity= -9.81,
                                 enable_wind= False,
                                 wind_power= 0.5,
                                 turbulence_power= 0.05)

state, info = env.reset(seed = SEED)
action = env.action_space.sample() 
state, reward, done, trun, info = env.step(action)

state_shape = env.observation_space.shape
state_size = state_shape[0]
action_size = env.action_space.n
print('State shape: ', state_size)
print('Number of actions: ', action_size)
print(action, round(reward, 3), done or trun)
plt.axis('off')
plt.imshow(pix_norm(env.render()))

Please refer to the instructions in `Deep_Q_Network.ipynb` if you would like to write your own DQN agent.  Otherwise, run the code cell below to load the solution files.

### 3. Train the Agent with DQN

Run the code cell below to train the agent from scratch.  You are welcome to amend the supplied values of the parameters in the function, to try to see if you can get better performance!

Alternatively, you can skip to the next step below (**4. Watch a Smart Agent!**), to load the saved model weights from a pre-trained agent.



In [ ]:
def dqn(n_episodes=2000, max_t=1000, eps_start=1.0, eps_end=0.01, eps_decay=0.995):
    """Deep Q-Learning.
    
    Params
    ======
        n_episodes (int): maximum number of training episodes
        max_t (int): maximum number of timesteps per episode
        eps_start (float): starting value of epsilon, for epsilon-greedy action selection
        eps_end (float): minimum value of epsilon
        eps_decay (float): multiplicative factor (per episode) for decreasing epsilon
    """
    FIRST = True
    episode_lengths = []
    scores = []                        # list containing scores from each episode
    window_size = 100                  # scores to rolling-remember
    scores_window = deque(maxlen=window_size)
    eps = eps_start                    # initialize epsilon
    for i_episode in range(1, n_episodes+1):
        state, _ = env.reset(seed=SEED)
        score = 0
        episteps = 0
        for t in range(max_t):  #episteps):
            action = agent.act(state, eps)
            next_state, reward, done, trun, _ = env.step(action)
            agent.step(state, action, reward, next_state, done or trun)
            state = next_state
            score += reward
            episteps += 1
            if done or trun:
                break 
        scores_window.append(score)       # save most recent score
        scores.append(score)              # save most recent score
        episode_lengths.append(episteps)
        eps = max(eps_end, eps_decay*eps) # decrease epsilon
        
        cycle_steps = agent.steps%BUFFER_SIZE
        buffer_cycle = agent.steps//BUFFER_SIZE

        print("\rEpisode {:4d} | Episode Score: {:7.2f} | Eps Steps: {:4d} | Epsilon: {:1.3f} | Buffer cycle:{:7d} +{:3d}".format(i_episode, 
                                                                                                                            scores_window[-1], 
                                                                                                                            episode_lengths[-1],
                                                                                                                            eps,
                                                                                                                            cycle_steps,
                                                                                                                            buffer_cycle), end="")
                                                                                                                                        
        if i_episode % 100 == 0:
            chkpntname = "data/chkpnt{}.pth".format(i_episode)
            torch.save(agent.qnetwork_local.state_dict(), chkpntname)   
            print("\rEpisode {:4d} | Average Score: {:7.2f} | Avg Steps: {:4d} | Epsilon: {:1.3f} | Buffer cycle:{:7d} +{:3d}".format(i_episode, 
                                                                                                                            np.mean(scores_window), 
                                                                                                                            round(np.mean(episode_lengths)),
                                                                                                                            eps,
                                                                                                                            cycle_steps,
                                                                                                                            buffer_cycle))
        #episteps = (episteps - 1) if episteps>=100 else max_t
        
        if np.mean(scores_window)>=200. and FIRST:
            print('\nEnvironment solved in {:d} episodes!\tAverage Score: {:5.2f}'.format(i_episode-100, np.mean(scores_window)))
            torch.save(agent.qnetwork_local.state_dict(), 'data/slvdpnt.pth')
            FIRST = False
        elif np.mean(scores_window)>=250. :
            print("\nHigh Score!")
            torch.save(agent.qnetwork_local.state_dict(), 'data/hipnt.pth')
            break
             
    return scores, episode_lengths

In [ ]:
### Initialize Agent 
agent = Agent(state_size=state_size, action_size=action_size, seed=SEED, 
                       fc1_units=256, fc2_units=128, learn_every=2)
scores = []; episode_lengths=[]; episode_times=[]

check_params = False
load_weights = False #True

if load_weights:
    try:
        agent.qnetwork_local.load_state_dict(torch.load('data/slvdpnt.pth'))#, map_location=torch.device('cpu')))
        agent.qnetwork_target.load_state_dict(torch.load('data/slvdpnt.pth'))#, map_location=torch.device('cpu')))
    except: print("Problem loading weights.")

if check_params:
    qnetwork_local_params = [p.detach().cpu().numpy() for p in agent.qnetwork_local.parameters()]
    qnetwork_target_params = [p.detach().cpu().numpy() for p in agent.qnetwork_target.parameters()]

    print(qnetwork_local_params[0]==qnetwork_target_params[0])
    print(qnetwork_local_params[1]==qnetwork_target_params[1])
    print(">> Qnetworks raw output")
    print("Local:", qnetwork_local_params[-1], "\nTarget:", qnetwork_target_params[-1])

In [ ]:
def train(n_episodes=2000, max_t=1000, eps_init=1.0, eps_decay=0.995, eps_end=0.001):
          #nu_init = 0.111, nu_decay=0.999, nu_end=0.011):
    """Deep Q-Learning but for DDPG    
    Params
    ======
        n_episodes (int): maximum number of training episodes
        max_t (int): maximum number of timesteps per episode
        eps_start (float): starting value of epsilon, for epsilon-greedy action selection
        eps_end (float): minimum value of epsilon
        eps_decay (float): multiplicative factor (per episode) for decreasing epsilon
        nu_decay (float): 
    """
    start = time.time()
    FIRST = True
    episode_times = []
    episode_lengths = []
    scores = []                        # list containing scores from each episode
    window_size = 100                  # scores to rolling-remember
    scores_window = deque(maxlen=window_size)
    eps = eps_init                    # initialize epsilon for greedy choices
    #nu = nu_init
    for i_episode in range(1, n_episodes+1):
        state, _ = env.reset(seed=SEED)
        score = 0
        episteps = 0
        epistart = time.time()
        for t in range(max_t): 
            action = agent.act(state, eps=eps)#, nu=nu)
            next_state, reward, done, trun, _ = env.step(action)
            agent.step(state, action, reward, next_state, done or trun)
            state = next_state
            score += reward
            episteps += 1
            if done or trun:
                break              
        episode_times.append(time.time()-epistart)
        scores_window.append(score)       
        scores.append(score)                 # save most recent score
        episode_lengths.append(episteps)
        eps = max(eps_end, eps*eps_decay)    # exponentially decay epsilon down to a specified minimum
        #nu = max(1e-5, nu*nu_decay)          # exponentially decay nu down to a minimum value
        
        cycle_steps = agent.steps%BUFFER_SIZE
        buffer_cycle = agent.steps//BUFFER_SIZE


        print("\r| Episode {} | Eps. Reward: {:6.2f} | Eps. Steps: {:4d} | Eps. Time: {:5.3f} sec | Epsilon: {:4.3f} | Buffer cycle:{:7d} +{:3d} |".format(i_episode,
                                                                                                                                            score, 
                                                                                                                                            episteps,
                                                                                                                                            time.time()-epistart,
                                                                                                                                            eps,
                                                                                                                                            cycle_steps,
                                                                                                                                            buffer_cycle), end="")

        if i_episode % 100 == 0:
            chkpntname = "data/qn_local_chkpnt{}.pth".format(i_episode)
            torch.save(agent.qnetwork_local.state_dict(), chkpntname)  
            chkpntname = "data/qn_target_chkpnt{}.pth".format(i_episode)
            torch.save(agent.qnetwork_target.state_dict(), chkpntname) 
            print("\r| {} Episodes | Avg. Reward: {:6.2f} | Avg. Steps: {:4d} | Avg. Time: {:5.3f} sec | Epsilon: {:4.3f} | Buffer cycle:{:7d} +{:3d} |".format(i_episode, 
                                                                                                                                        np.mean(scores_window), 
                                                                                                                                        round(np.mean(episode_lengths)),
                                                                                                                                        np.mean(episode_times),
                                                                                                                                        eps,
                                                                                                                                        cycle_steps,
                                                                                                                                        buffer_cycle))
        if np.mean(scores_window)>=200. and FIRST:
            print('\nEnvironment solved after {:d} episodes!\tAverage Score: {:5.2f}'.format(i_episode-100, np.mean(scores_window)))
            torch.save(agent.qnetwork_local.state_dict(), 'data/local_slvdpnt.pth')
            torch.save(agent.qnetwork_target.state_dict(), 'data/target_slvdpnt.pth')
            FIRST = False
        elif np.mean(scores_window)>=250. :
            print("\n***** High Score! *****")
            print("\tGame over.")
            torch.save(agent.qnetwork_local.state_dict(), 'data/local_hipnt.pth')
            torch.save(agent.qnetwork_target.state_dict(), 'data/target_hipnt.pth')
            break
            
    t_time = time.time() - start
    print("Total time: {:3d} minutes {:4.2f} seconds \t Avg. Episode time: {:5.3f} seconds \t Avg. Steps per Episode: {:5.1f}".format(int(t_time//60),
                                                                                                                                   t_time%60,
                                                                                                                                   t_time/i_episode,
                                                                                                                                   round(np.mean(episode_lengths))))
    return scores, episode_lengths, episode_times

In [ ]:
############### Train   
n_episodes= 200
max_t=      900
eps_init=   1.0
eps_decay=  0.995
eps_end=    1e-2
#nu_init=    0.22
#nu_decay=   0.997
#nu_end=     0.011
print('\r### Episodes: {}\t###\tStep Limit: {} ###'.format(n_episodes, max_t))

new_scores, new_els, new_ets = train(n_episodes=n_episodes, max_t=max_t, 
                                     eps_init=eps_init, eps_decay=eps_decay, eps_end=eps_end)#, 
                                     #nu_init=nu_init, nu_decay=nu_decay, nu_end=nu_end)
    
scores += new_scores; episode_lengths += new_els ; episode_times += new_ets

`Episode  100 | Average Score: -254.49 | Avg Steps:  118 | Epsilon: 0.606 | Buffer cycle:  11827 +  0

Episode  200 | Average Score: -104.40 | Avg Steps:  144 | Epsilon: 0.367 | Buffer cycle:  28800 +  0

Episode  300 | Average Score: -101.45 | Avg Steps:  223 | Epsilon: 0.222 | Buffer cycle:  66887 +  0

Episode  400 | Average Score:  -17.39 | Avg Steps:  402 | Epsilon: 0.135 | Buffer cycle:  60691 +  1

Episode  500 | Average Score:   36.90 | Avg Steps:  483 | Epsilon: 0.082 | Buffer cycle:  41366 +  2

Episode  600 | Average Score:   -2.30 | Avg Steps:  561 | Epsilon: 0.049 | Buffer cycle:  36642 +  3

Episode  700 | Average Score:  -11.75 | Avg Steps:  620 | Epsilon: 0.030 | Buffer cycle:  33737 +  4

Episode  800 | Average Score:   -0.93 | Avg Steps:  650 | Epsilon: 0.018 | Buffer cycle:  19645 +  5

Episode  900 | Average Score:   39.81 | Avg Steps:  678 | Epsilon: 0.011 | Buffer cycle:  10402 +  6

Episode 1000 | Average Score:  140.54 | Avg Steps:  673 | Epsilon: 0.010 | Buffer cycle:  73226 +  6

Episode 1100 | Average Score:  162.32 | Avg Steps:  652 | Epsilon: 0.010 | Buffer cycle:  17503 +  7

Episode 1166 | Episode Score:  258.97 | Eps Steps:  464 | Epsilon: 0.010 | Buffer cycle:  46212 +  7

Environment solved in 1066 episodes!	Average Score: 201.19

Episode 1200 | Average Score:  218.04 | Avg Steps:  635 | Epsilon: 0.010 | Buffer cycle:  61530 +  7`

In [ ]:
experiences = agent.memory.memory
states, actions, rewards, next_states, dones = disentangle(experiences)
states = np.asarray(states)
next_states = np.asarray(next_states)
rewards = np.asarray(rewards)
actions = np.asarray(actions)

In [ ]:
#torch.save(agent.actor_local.state_dict(), 'data/checkpoint_actor.pth')
#torch.save(agent.critic_local.state_dict(), 'data/checkpoint_critic.pth'))
#agent.actor_local.load_state_dict(torch.load('data/actor_chkpnt300.pth'))
#agent.critic_local.load_state_dict(torch.load('data/critic_chkpnt300.pth'))

In [ ]:
if True:
    state = torch.from_numpy(states[9117]).float().unsqueeze(0).to(device)
    agent.qnetwork_local.eval()
    with torch.no_grad():
        # Action values are raw outputs of network (ie. no activation function for last layer)
        action_values = agent.qnetwork_local(state)
    agent.qnetwork_local.train()
    action_values = action_values.cpu().squeeze().numpy()
    print(np.argmax(action_values))
    print(action_values)

In [ ]:
if True:
    # plot the scores
    fig = plt.figure(figsize=(12,4))
    ax = fig.add_subplot(111)
    plt.plot(np.arange(len(scores)), scores, 'b.--', linewidth=0.5, markersize=1.0)
    plt.ylabel('Score')
    plt.xlabel('Episode #')
    #plt.xlim(0,25)
    plt.show()
else:
    # plot the normed scores
    norm = lambda x: (x - np.mean(x))/np.std(x) if np.std(x)!=0. else 0.
    fig = plt.figure(figsize=(12,4))
    ax = fig.add_subplot(111)
    plt.plot(np.arange(len(scores)), norm(np.asarray(scores)), 'c,--', linewidth=0.5, markersize=1.0,)
    plt.ylabel('Normed Score')
    plt.xlabel('Episode #')
    plt.show()

In [ ]:
#### Display from DQN notebook
#chkpnt_name = "data/slvdpnt.pth"#"data/chkpnt{}.pth".format(chkpnt)  #
#agent.qnetwork_local.load_state_dict(torch.load(chkpnt_name))

# load the weights from file
#agent.qnetwork_local.load_state_dict(torch.load('checkpoint.pth'))

# watch a pre-trained agent
f = plt.figure(figsize=(10,4))
n=4
for i in range(n):
#for chkpnt in [100, 300, 500, 700, 1000]:
    #chkpnt_name = "data/actor_chkpnt{}.pth".format(chkpnt)
    #agent.actor_local.load_state_dict(torch.load(chkpnt_name))
    #chkpnt_name = "data/critic_chkpnt{}.pth".format(chkpnt)
    #agent.critic_local.load_state_dict(torch.load(chkpnt_name))
    score = 0
    state, _ = env.reset()
    img = plt.imshow(env.render())
    for j in range(1000):
        action = agent.act(state)
        state, reward, done, trun, _ = env.step(action)
        score += reward
        img.set_data(env.render()) 
        plt.axis('off')
        #title = "{} | Steps: {:4d} | R: {:7.2f} | M: {:3.2f}  S: {:3.2f}".format(chkpnt, j+1, score, action[0], action[1])
        title = "{} of {} | Step: {:4d} | Sum_Rs: {:7.2f} | Action: {:5s} ".format(i+1, n, j+1, score, num2act[action])
        plt.title(title)
        display.display(plt.gcf())
        display.clear_output(wait=True)
        if done or trun:
            break 
print("Total episode reward: ", np.round(np.sum(score), 3))

In [ ]:
# plot the episode_steps
#norm = lambda x: (x - np.mean(x))/np.std(x) if np.std(x)!=0. else 0.
score_per_step = [s/el for s, el in zip(scores,episode_lengths)]
fig = plt.figure(figsize=(12,4))
ax = fig.add_subplot(111)
plt.plot(np.arange(len(episode_lengths)), score_per_step, 'go--', linewidth=0.75, markersize=1.0,)
plt.plot(np.arange(len(scores)), norm(np.asarray(episode_lengths)), 'bo--', linewidth=0.25, markersize=1.0,)
plt.plot(np.arange(len(scores)), norm(np.asarray(scores)), 'r,--', linewidth=0.25, markersize=1.0,)
plt.ylabel('Score / # Episode Steps')
plt.xlabel('Episode #')
plt.legend(["R/step", "Steps", "Rewards"])
plt.show()

In [ ]:
len(actions[::5]), len(x)#[::5])
#actions, x

In [ ]:
### Actions plot

#main_act= [a[0] for a in actions]

dim0= len(actions)
x = np.arange(0, dim0)
lim = dim0   #15000 #
xlims = (0, lim)
mylims = ( round(min(actions[:lim]), 2), round(max(actions[:lim]), 2) )
print(mylims)

plots=[]
fig = plt.figure(figsize=(12, 10))

ax1 = fig.add_subplot(311)
plt.ylabel('Main and Side Engine Values')
ax1.scatter(x, actions, alpha=0.667, c='#0ab', s=0.4)
#ax1.scatter(x, side_act, alpha=0.4, c='#ba0', s=0.4)
plt.title("Beginning of training")
plt.legend(["main engine","side engine"], loc='upper right')
plt.ylim(*mylims)
plt.xlim(0, 7500)

ax2 = fig.add_subplot(312)
plt.ylabel('Main and Side Engine Values')
plt.xlabel('Steps (s)')
ax2.scatter(x, actions, alpha=0.7, c='#f4a', s=0.6)
#ax2.scatter(x, side_act, alpha=0.5, c='#a4f', s=0.6)
plt.title("End of training")
plt.legend(["main engine","side engine"], loc='upper right')
plt.ylim(*mylims)
plt.xlim(lim-7500, lim)

ax3 = fig.add_subplot(313)
plt.ylabel('Main and Side Engine Values')
plt.xlabel('Steps (s)')
ax3.scatter(x, actions, alpha=0.6, c='#12e', s=0.25)
#ax3.scatter(x, side_act, alpha=0.4, c='#dd3', s=0.25)
plt.title("Entire training set")
plt.legend(["main engine","side engine"], loc='upper right')
plt.ylim(*mylims)

plt.show()

In [ ]:
### Actions plot

#act=[(a[0],a[1]) for a in action_noise[:,0]]
#ion=[(n[0],n[1]) for n in action_noise[:,1]]

main_act= [a[0] for a in actions]
#main_ion= [n[0] for n in ion]
side_act= [a[1] for a in actions]
#side_ion= [n[1] for n in ion]

dim0= len(actions)
x = np.arange(0, dim0)
lim = dim0   #15000 #
xlims = (0, lim)
mylims = ( round(min(main_act[:lim] + side_act[:lim]), 2), round(max(main_act[:lim] + side_act[:lim]), 2) )
#mylims = ( round(min(main_act[:lim]), 2), round(max(main_act[:lim]), 2) )
print(mylims)
#sylims = ( round(min(side_act[:lim]), 2), round(max(side_act[:lim]), 2) )
#print(sylims)

plots=[]
fig = plt.figure(figsize=(12, 12))

ax1 = fig.add_subplot(311)
plt.ylabel('Main and Side Engine Values')
ax1.scatter(x, main_act, alpha=0.667, c='#0ab', s=0.4)#, linewidth=0.1, markersize=1.75
ax1.scatter(x, side_act, alpha=0.4, c='#ba0', s=0.4)#, linewidth=0.1, markersize=1.75
plt.title("Beginning of training")
plt.legend(["main engine","side engine"])
plt.ylim(*mylims)
plt.xlim(0, 12000)

ax2 = fig.add_subplot(312)
plt.ylabel('Main and Side Engine Values')
plt.xlabel('Steps (s)')
ax2.scatter(x, main_act, alpha=0.7, c='#f4a', s=0.6)
ax2.scatter(x, side_act, alpha=0.5, c='#a4f', s=0.6)
plt.title("End of training")
plt.legend(["main engine","side engine"])
plt.ylim(*mylims)
plt.xlim(lim-12000, lim)

ax3 = fig.add_subplot(313)
plt.ylabel('Main and Side Engine Values')
plt.xlabel('Steps (s)')
ax3.scatter(x, main_act, alpha=0.6, c='#12e', s=0.25)
ax3.scatter(x, side_act, alpha=0.4, c='#dd3', s=0.25)
plt.title("Entire training set")
plt.legend(["main engine","side engine"])
plt.ylim(*mylims)

plt.show()

In [ ]:
#np.mean(states), np.std(states), np.min(states), np.max(states), states.shape
#np.mean(actions), np.std(actions), np.min(actions), np.max(actions), actions.shape
np.mean(rewards), np.std(rewards), np.min(rewards), np.max(rewards), rewards.shape


In [ ]:
# plot the states
lims = (18000,22000)
plots = []
fig = plt.figure(figsize=(12,5))

ax1 = fig.add_subplot(211)
plt.ylabel('States (high/low)')
plt.xlabel('Steps (s)')
plt.xlim(*lims)
#xlim(80000, 100000)
#ax1.sharey(ax2)
plots.append(ax1.plot(np.arange(len(states)), states, linewidth=0.5, markersize=0.2, alpha=0.75))

ax2 = fig.add_subplot(212)
plt.ylabel('Next States (high/low)')
plt.xlabel('Steps (s)')
plt.xlim(*lims)
#ax2.sharey(ax1)
plots.append(ax2.plot(np.arange(len(next_states)), next_states, linewidth=0.5, markersize=0.2, alpha=0.75))


#plt.show()

### 4. Watch a Smart Agent!

In the next code cell, you will load the trained weights from file to watch a smart agent!

In [ ]:
# load the weights from file
#agent.qnetwork_local.load_state_dict(torch.load('checkpoint.pth'))

# watch a pre-trained agent

#for chkpnt in range(100,1300,300):
for chkpnt in [100,400,800]:    
    chkpnt_name = "data/chkpnt{}.pth".format(chkpnt)
    agent.qnetwork_local.load_state_dict(torch.load(chkpnt_name))
    score = 0
    state, _ = env.reset()
    img = plt.imshow(env.render())
    for j in range(1000):
        action = agent.act(state)
        state, reward, done, trun, _ = env.step(action)
        score += reward
        img.set_data(env.render()) 
        plt.axis('off')
        title = "{:4d} | {:5d} | {:7.2f} | {:6s}".format(chkpnt, j+1, score, num2act[action])
        plt.title(title)
        display.display(plt.gcf())
        display.clear_output(wait=True)
        if done or trun:
            break 

#env.close()

In [ ]:
chkpnt_name = "data/local_hipnt.pth"#"data/chkpnt{}.pth".format(chkpnt)  #
agent.qnetwork_local.load_state_dict(torch.load(chkpnt_name))
score = 0
state, _ = env.reset()
img = plt.imshow(env.render())
for j in range(1000):
    action = agent.act(state)
    state, reward, done, trun, _ = env.step(action)
    score += reward
    img.set_data(env.render()) 
    plt.axis('off')
    title = "{:6s} | {:5d} | {:8.2f} | {:7s}".format("Solved", j+1, score, num2act[action])
    plt.title(title)
    display.display(plt.gcf())
    display.clear_output(wait=True)
    if done or trun:
        break 

### 5. Explore

In this exercise, you have implemented a DQN agent and demonstrated how to use it to solve an OpenAI Gym environment.  To continue your learning, you are encouraged to complete any (or all!) of the following tasks:
- Amend the various hyperparameters and network architecture to see if you can get your agent to solve the environment faster.  Once you build intuition for the hyperparameters that work well with this environment, try solving a different OpenAI Gym task with discrete actions!
- You may like to implement some improvements such as prioritized experience replay, Double DQN, or Dueling DQN! 
- Write a blog post explaining the intuition behind the DQN algorithm and demonstrating how to use it to solve an RL environment of your choosing.  

In [ ]:
def DELETE_dqn(n_episodes=2000, max_t=1000, eps_start=1.0, eps_end=0.01, eps_decay=0.995):
    """Deep Q-Learning.
    
    Params
    ======
        n_episodes (int): maximum number of training episodes
        max_t (int): maximum number of timesteps per episode
        eps_start (float): starting value of epsilon, for epsilon-greedy action selection
        eps_end (float): minimum value of epsilon
        eps_decay (float): multiplicative factor (per episode) for decreasing epsilon
    """
    scores = []                        # list containing scores from each episode
    scores_window = deque(maxlen=100)  # last 100 scores
    eps = eps_start                    # initialize epsilon
    for i_episode in range(1, n_episodes+1):
        state, _ = env.reset()
        score = 0
        for t in range(max_t):
            action = agent.act(state, eps)
            next_state, reward, done, trun, _ = env.step(action)
            agent.step(state, action, reward, next_state, done or trun)
            state = next_state
            score += reward
            if done or trun:
                break 
        scores_window.append(score)       # save most recent score
        scores.append(score)              # save most recent score
        eps = max(eps_end, eps_decay*eps) # decrease epsilon
        print('\rEpisode {}\tAverage Score: {:.2f}'.format(i_episode, np.mean(scores_window)), end="")
        if i_episode % 100 == 0:
            print('\rEpisode {}\tAverage Score: {:.2f}'.format(i_episode, np.mean(scores_window)))
            chkpnt_name = "data/target_chkpnt{}.pth".format(i_episode)
            torch.save(agent.qnetwork_target.state_dict(), chkpnt_name)
        if np.mean(scores_window)>=200.0:
            print('\nEnvironment solved in {:d} episodes!\tAverage Score: {:.2f}'.format(i_episode-100, np.mean(scores_window)))
            torch.save(agent.qnetwork_local.state_dict(), 'data/solution_chkpnt.pth')
            break
    return scores



In [ ]:
for i in range(10):
    print(random.sample(range(5), k=4))